# 🏨 **Hotel Cancel Culture: Who Will Cancel?** 🏨

**Who?**
>* 🏢 **Revenue Management (RM) teams** for hotel groups (corporate, franchise)
>
>
>* 🏨 On-site GMs, Sales, and Ops teams

**Why?**
>* 💰 **Revenue Management:** 
>  * Revenue optimization: Right price, right time, right customer
>    * Dynamic pricing
>    * Distribution channels
>    * Pricing per room type
>
>
>* 🤝 **Sales:**
>  * Group sales (pickup/wash)
>  * BT (performance/company for both GPP and LNR rates)
>
>
>* 🛌 **Rooms Ops:**
>  * Forecasting occupancy, arrivals, departures, stay-overs, same-day booking demand, and probability of guest relocation in the case of oversell.
>  * Determining staff schedules and periods of high demand
>
>
>* 🍰 ☕ **Food and Beverage:**
>  * Ordering food/supplies overall
>  * Scheduling staff
>  * Determining busy times (breakfast, lunch, dinner)
>    * Staffing, specific food/supplies

**What?**
>* 🧾 Dataset comprised of... 
>  * 32 different features
>    * Detailed explanation of features (and sub-categories, when appropriate) available in Readme
>  * Nearly 120,000 reservation records
>  * Source cited in Readme

❌ **How?**
>* Which models/methods? 
>* Data prep and feature engineering

**Goal:** determine whether or not a reservation would cancel given different details of a guest and their reservation.

# 📌 **To-Do**

>* Update each section with ***working*** code
>* Include references/details for relevant packages
---

- [ ] [TD1](#td1)
- [ ] [TD2](#td2)
- [ ] [TD3](#td3)
- [ ] [todo4](#td4)
- [ ] [todo5](#td5)
- [ ] [todo6](#td6)
- [ ] [todo7](#td7)

# 📂 **Imports**

In [1]:
# Data Handling
import pandas as pd
import numpy as np
from scipy import stats

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# Modeling - SKLearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

# %load_ext autoreload
# %autoreload 2

import hotel_data_functions.functions as hdf

# from sklearn.naive_bayes import MultinomialNB # for naive bayes model

# Settings
%matplotlib inline
plt.style.use('seaborn-talk')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

from sklearn import set_config
set_config(display='diagram')

# 📖 **Read Data**

In [2]:
## Reading data and saving to a DataFrame
df = pd.read_csv('hotel_bookings.csv')
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,nan,nan,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,nan,nan,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,nan,nan,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.00,nan,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.00,nan,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0.00,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.00,nan,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0.00,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.00,nan,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0.00,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.00,nan,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.00,nan,0,Transient,104.40,0,0,Check-Out,2017-09-07


In [3]:
## Creating features/target for dataset
target = 'is_canceled'

X = df.drop(target, axis=1).copy()
y = df[target].copy()

In [4]:
## Confirming same size of rows
X.shape[0] == y.shape[0]

True

In [5]:
## Splitting to prevent data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 🧼 **Starting Cleaning and EDA**

In [6]:
## Checking for class imbalances
y_train.value_counts()

0    56446
1    33096
Name: is_canceled, dtype: int64

> Classes are not dramatically out of balance in the prediction data for the training set. I will not need to perform any over-sampling for this data.

From original article:

>"The PMS assured no missing data exists in its database tables. However, in some categorical variables like Agent or Company, **“NULL”** is presented as one of the categories. This **should not be considered a missing value, but rather as “not applicable”.** For example, if a booking “Agent” is defined as “NULL” it means that the booking did not came from a travel agent."

This indicates that "NULL" values are a distinct category in themselves, and I will need to treat them as a category instead of treating as a NaN/null/missing value.

In [7]:
hdf.report_df(X_train)

(89542, 31)


,datatypes,num_unique,null_sum,null_pct,count,mean,std,min,25%,50%,75%,max
hotel,object,2,0,0.00,nan,nan,nan,nan,nan,nan,nan,nan
lead_time,int64,476,0,0.00,"89,542.00",103.84,106.88,0.00,18.00,69.00,160.00,737.00
arrival_date_year,int64,3,0,0.00,"89,542.00","2,016.16",0.71,"2,015.00","2,016.00","2,016.00","2,017.00","2,017.00"
arrival_date_month,object,12,0,0.00,nan,nan,nan,nan,nan,nan,nan,nan
arrival_date_week_number,int64,53,0,0.00,"89,542.00",27.15,13.60,1.00,16.00,27.00,38.00,53.00
arrival_date_day_of_month,int64,31,0,0.00,"89,542.00",15.79,8.79,1.00,8.00,16.00,23.00,31.00
stays_in_weekend_nights,int64,16,0,0.00,"89,542.00",0.93,1.00,0.00,0.00,1.00,2.00,18.00
stays_in_week_nights,int64,32,0,0.00,"89,542.00",2.50,1.91,0.00,1.00,2.00,3.00,42.00
adults,int64,12,0,0.00,"89,542.00",1.86,0.58,0.00,2.00,2.00,2.00,50.00
children,float64,5,4,0.00,"89,538.00",0.10,0.40,0.00,0.00,0.00,0.00,10.00


## ❓ **Null Values and How to Address:**

>* **Children:** fill in missing values with zero (75% of the data is zero, making it an easy and accurate choice)
>
>
>* **Country, Agent:** For these three sections, the documentation for the dataset indicates that "NULL" or NaN values are not truly *missing* from the dataset, but that the data source did not have any values for those reservations.
>  * **To address:** I will impute the string, "missing," as a placeholder to keep these features in the dataset.
>
>
>* **Company:** Since this feature is 94% empty values, it would have an insignificant impact on the predictions of whether or not a guest will cancel their reservation.
>  * **To address:** I will drop the column as part of the 

In [8]:
X_train.head()

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
118418,City Hotel,208,2017,August,33,17,0,3,2,0.00,0,BB,DEU,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.00,nan,0,Transient,131.70,1,0,Check-Out,2017-08-20
66950,City Hotel,110,2017,April,17,26,1,4,2,0.00,0,SC,NOR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.00,nan,0,Transient,100.80,0,0,Canceled,2017-01-06
52240,City Hotel,330,2016,June,23,2,0,2,2,0.00,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,1.00,nan,0,Transient,62.00,0,0,Canceled,2015-10-21
3835,Resort Hotel,3,2016,January,4,21,0,1,1,0.00,0,BB,PRT,Corporate,Corporate,0,0,0,A,E,0,No Deposit,nan,47.00,0,Transient,30.00,0,0,Canceled,2016-01-18
3330,Resort Hotel,66,2015,December,49,5,1,1,2,0.00,0,FB,PRT,Groups,TA/TO,0,0,0,A,A,0,No Deposit,155.00,nan,0,Transient-Party,64.00,0,0,Canceled,2015-12-02


In [9]:
X_train['meal'].value_counts()

BB           69242
HB           10794
SC            8027
Undefined      888
FB             591
Name: meal, dtype: int64

In [10]:
und_count = X_train.isin(['Undefined']).sum()
# und_count

In [11]:
und_count[und_count > 0]

meal                    888
market_segment            2
distribution_channel      4
dtype: int64

#### Old code - saving just in case

In [12]:
## Unsure what this code was for - something about the "Undefined" values? 

# for col in X_train.columns:
#     if X_train[col].dtype == 'O':
#         print(X_train[col].value_counts(),'\n\n')

In [13]:
# for col in X_train.columns:
#     if 'Undefined' in X_train[col]:
#         print(col)

In [14]:
# 'undefined' in X_train["meal"]

In [15]:
# X_train['distribution_channel'].unique()

In [16]:
# #3 Identifying which columns indicate missing values based on .isna() method

# nan_cols = []

# for column in X_train.columns:
#     if X_train[column].isna().sum() > 0:
#         nan_cols.append(column)
#         print(column,'\n\n',X_train[column].value_counts(dropna=False),'\n\n')
        
# nan_cols

In [17]:
# for column in X_train.columns:
#     if X_train[column].isna().sum() > 0:
# #         nan_cols.append(column)
#         print(column,'\n\n',X_train[column].isna().sum(),'\n\n')

In [18]:
# X_train['country'].isna().sum()

In [19]:
# X_train[X_train['country'].isna()]

# 🚿 **Preprocessing Pipeline**

In [20]:
cat_cols = ['hotel', 'meal','arrival_date_month', 'country', 'market_segment',
            'distribution_channel','is_repeated_guest','reserved_room_type',
            'assigned_room_type','deposit_type', 'agent',
            'customer_type','reservation_status']

cont_cols = [col for col in X_train.drop(['reservation_status_date','company'],axis=1).columns if col not in cat_cols]

cont_cols

['lead_time',
 'arrival_date_year',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests']

In [21]:
X_train[cat_cols] = X_train[cat_cols].astype(str)

C:\Users\bmcca\anaconda3\envs\learn-env-bmc\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [22]:
X_test[cat_cols] = X_test[cat_cols].astype(str)

In [23]:
## Creating ColumnTransformer and sub-transformers for imputation and encoding

# Filling missing "Children"
zero_transformer = SimpleImputer(strategy='constant', fill_value=0)

##  
missing_transformer = SimpleImputer(strategy='constant', fill_value='missing')

## Encoding categoricals - handling errors to prevent issues w/ test set
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

cat_pipe = Pipeline(steps=[('imputer', missing_transformer),
                      ('ohe', categorical_transformer)])

cont_pipe = Pipeline(steps=[('imputer', zero_transformer),
                           ('scaler', StandardScaler())])

## Instantiating the ColumnTransformer and including all transformers
preprocessor = ColumnTransformer(
    transformers=[('conts', cont_pipe, cont_cols),
                  ('cats', cat_pipe, cat_cols)])

preprocessor

ColumnTransformer(transformers=[('conts',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant')),
                                                 ('scaler', StandardScaler())]),
                                 ['lead_time', 'arrival_date_year',
                                  'arrival_date_week_number',
                                  'arrival_date_day_of_month',
                                  'stays_in_weekend_nights',
                                  'stays_in_week_nights', 'adults', 'children',
                                  'babies', 'previous_cancellations',
                                  'previous_bo...
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['hotel', 'meal', 'arrival_date_month',
                                  'country', 'market_segment',
                                  'distribution_channel', 'is_repeated_guest',
                                  'reserved_room_type', 'assigned_room_type',
                                  'deposit_type', 'agent', 'customer_type',
                                  'reservation_status'])])

In [24]:
preprocessor.fit(X_train)

## Getting feature names from OHE
ohe_cat_names = preprocessor.named_transformers_['cats'].named_steps['ohe'].get_feature_names(cat_cols)

In [25]:
## Generating list for column index
final_cols = [*cont_cols, *ohe_cat_names]

## Fit and transform the data via the ColumnTransformer
X_train_tf = preprocessor.transform(X_train)
X_train_tf_df = pd.DataFrame(X_train_tf, columns=final_cols, index=X_train.index)

## Transforming the test set and saving
X_test_tf = preprocessor.transform(X_test)
X_test_tf_df = pd.DataFrame(X_test_tf, columns=final_cols, index=X_test.index)

display(X_train_tf_df.head(5),X_test_tf_df.head(5))

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,hotel_City Hotel,hotel_Resort Hotel,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,arrival_date_month_March,arrival_date_month_May,arrival_date_month_November,arrival_date_month_October,arrival_date_month_September,country_ABW,country_AGO,country_AIA,country_ALB,country_AND,country_ARE,country_ARG,country_ARM,country_ASM,country_ATA,country_ATF,country_AUS,country_AUT,country_AZE,country_BDI,country_BEL,country_BEN,country_BFA,country_BGD,country_BGR,country_BHR,country_BHS,country_BIH,country_BLR,country_BOL,country_BRA,country_BRB,country_BWA,country_CAF,country_CHE,country_CHL,country_CHN,country_CIV,country_CMR,country_CN,country_COL,country_COM,country_CPV,country_CRI,country_CUB,country_CYM,country_CYP,country_CZE,country_DEU,country_DMA,country_DNK,country_DOM,country_DZA,country_ECU,country_EGY,country_ESP,country_EST,country_ETH,country_FIN,country_FJI,country_FRA,country_FRO,country_GAB,country_GBR,country_GEO,country_GGY,country_GHA,country_GIB,country_GLP,country_GNB,country_GRC,country_GTM,country_GUY,country_HKG,country_HRV,country_HUN,country_IDN,country_IMN,country_IND,country_IRL,country_IRN,country_IRQ,country_ISL,country_ISR,country_ITA,country_JAM,country_JEY,country_JOR,country_JPN,country_KAZ,country_KEN,country_KHM,country_KIR,country_KNA,country_KOR,country_KWT,country_LAO,country_LBN,country_LBY,country_LIE,country_LKA,country_LTU,country_LUX,country_LVA,country_MAC,country_MAR,country_MCO,country_MDG,country_MDV,country_MEX,country_MKD,country_MLI,country_MLT,country_MMR,country_MNE,country_MOZ,country_MUS,country_MWI,country_MYS,country_NAM,country_NCL,country_NGA,country_NIC,country_NLD,country_NOR,country_NPL,country_NZL,country_OMN,country_PAK,country_PAN,country_PER,country_PHL,country_PLW,country_POL,country_PRI,country_PRT,country_PRY,country_PYF,country_QAT,country_ROU,country_RUS,country_RWA,country_SAU,country_SDN,country_SEN,country_SGP,country_SLE,country_SLV,country_SMR,country_SRB,country_STP,country_SUR,country_SVK,country_SVN,country_SWE,country_SYC,country_SYR,country_TGO,country_THA,country_TJK,country_TMP,country_TUN,country_TUR,country_TWN,country_TZA,country_UGA,country_UKR,country_UMI,country_URY,country_USA,country_UZB,country_VEN,country_VGB,country_VNM,country_ZAF,country_ZMB,country_ZWE,country_nan,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,is_repeated_guest_0,is_repeated_guest_1,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,agent_1.0,agent_10.0,agent_103.0,agent_104.0,agent_105.0,agent_106.0,agent_107.0,agent_11.0,agent_110.0,agent_111.0,agent_112.0,agent_114.0,agent_115.0,agent_118.0,agent_119.0,agent_12.0,agent_121.0,agent_126.0,agent_127.0,agent_128.0,agent_129.0,agent_13.0,agent_132.0,agent_133.0,agent_134.0,agent_135.0,agent_138.0,agent_139.0,agent

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,hotel_City Hotel,hotel_Resort Hotel,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,arrival_date_month_March,arrival_date_month_May,arrival_date_month_November,arrival_date_month_October,arrival_date_month_September,country_ABW,country_AGO,country_AIA,country_ALB,country_AND,country_ARE,country_ARG,country_ARM,country_ASM,country_ATA,country_ATF,country_AUS,country_AUT,country_AZE,country_BDI,country_BEL,country_BEN,country_BFA,country_BGD,country_BGR,country_BHR,country_BHS,country_BIH,country_BLR,country_BOL,country_BRA,country_BRB,country_BWA,country_CAF,country_CHE,country_CHL,country_CHN,country_CIV,country_CMR,country_CN,country_COL,country_COM,country_CPV,country_CRI,country_CUB,country_CYM,country_CYP,country_CZE,country_DEU,country_DMA,country_DNK,country_DOM,country_DZA,country_ECU,country_EGY,country_ESP,country_EST,country_ETH,country_FIN,country_FJI,country_FRA,country_FRO,country_GAB,country_GBR,country_GEO,country_GGY,country_GHA,country_GIB,country_GLP,country_GNB,country_GRC,country_GTM,country_GUY,country_HKG,country_HRV,country_HUN,country_IDN,country_IMN,country_IND,country_IRL,country_IRN,country_IRQ,country_ISL,country_ISR,country_ITA,country_JAM,country_JEY,country_JOR,country_JPN,country_KAZ,country_KEN,country_KHM,country_KIR,country_KNA,country_KOR,country_KWT,country_LAO,country_LBN,country_LBY,country_LIE,country_LKA,country_LTU,country_LUX,country_LVA,country_MAC,country_MAR,country_MCO,country_MDG,country_MDV,country_MEX,country_MKD,country_MLI,country_MLT,country_MMR,country_MNE,country_MOZ,country_MUS,country_MWI,country_MYS,country_NAM,country_NCL,country_NGA,country_NIC,country_NLD,country_NOR,country_NPL,country_NZL,country_OMN,country_PAK,country_PAN,country_PER,country_PHL,country_PLW,country_POL,country_PRI,country_PRT,country_PRY,country_PYF,country_QAT,country_ROU,country_RUS,country_RWA,country_SAU,country_SDN,country_SEN,country_SGP,country_SLE,country_SLV,country_SMR,country_SRB,country_STP,country_SUR,country_SVK,country_SVN,country_SWE,country_SYC,country_SYR,country_TGO,country_THA,country_TJK,country_TMP,country_TUN,country_TUR,country_TWN,country_TZA,country_UGA,country_UKR,country_UMI,country_URY,country_USA,country_UZB,country_VEN,country_VGB,country_VNM,country_ZAF,country_ZMB,country_ZWE,country_nan,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,is_repeated_guest_0,is_repeated_guest_1,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,agent_1.0,agent_10.0,agent_103.0,agent_104.0,agent_105.0,agent_106.0,agent_107.0,agent_11.0,agent_110.0,agent_111.0,agent_112.0,agent_114.0,agent_115.0,agent_118.0,agent_119.0,agent_12.0,agent_121.0,agent_126.0,agent_127.0,agent_128.0,agent_129.0,agent_13.0,agent_132.0,agent_133.0,agent_134.0,agent_135.0,agent_138.0,agent_139.0,agent

# 📝 Next Steps

* Process classification model - i.e. Logreg, KNN, DecisionTrees, etc.
* Evaluate results
* Determine if I need to redo pre-processing steps

# 🚿 Classification Pipeline